# 02. User Animelist Dataset

概要を確認する事ができたので、このページでは、分析を行うために必要なデータの整形を行っていく。

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

`ratings`テーブルのメモリ使用料は、およそ3GBもある事が分かっている。

さらに、`animes`テーブルの`genres`、`genres_detailed`は、複数のジャンルがPythonのリスト形式で格納されている形であり、これをそのまま展開してしまうとと、情報量が無尽蔵に増えてしまい、処理を行えなくなってしまう。

そこで、`ratings`テーブルに、分析に最低限必要な情報をマージしたテーブルと、個々のアニメの情報を参照するためのテーブルを別途作成する方針を取る事にする。

## データの軽量化と整形
まずは、`ratings`テーブルに、必要な情報を付加する作業を行っていく。<br>
`ratings`テーブルはそのままでは情報量が膨大なので、まずは、データを軽くするための工程を行っていく。

#### データフレームの目盛り使用料を確認する関数
`memory_usage`メソッドでメモリ使用量を表示可能。<br>
`deep=True`を付けることで、文字列やオブジェクト型の実際の使用メモリも正確に計算できる。

メモリ使用量をMBで表示する関数。

In [2]:
def display_MB(df):
    return f"{df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB"

GBで表示する関数。

In [3]:
def display_GB(df):
    return f"{df.memory_usage(deep=True).sum() / (1024 ** 3):.2f} GB"

### 軽量化作業

軽量化の方法として、まずは読み込み段階でデータを絞り込み、さらにより軽量なデータ型を指定する。

In [4]:
# 軽量化処理済の、読み込み関数
animes = pd.read_csv(
    Path("..", "data", "raw", "animes.csv"),
    usecols=[
        'animeID', 'type', 'year', 'episodes', 'sequel',
    ],
    dtype={
        'animeID': "int16",
        'type': "category",
        'year': "object",  # "?"が含まれているため
        'episodes': "int16",
        'sequel': "bool",
    }
)

In [5]:
display_MB(animes)

'1.14 MB'

`usecols`引数で抽出する列を指定、さらに`dtype`引数でそれぞれの列の型を明示してメモリを節約する。<br>
これで、マージ用の`animes`テーブルの軽量化に成功した。

`ratings`テーブルでも軽量化を試みてみる。

In [6]:
ratings = pd.read_csv(
    Path("..", "data", "raw", "ratings.csv"),
    dtype={
        "userID": "int32",
        "animeID": "int16",
        "rating": "int8"
    }
)

In [7]:
display_GB(ratings)

'0.97 GB'

データ型を小さいものにするだけで、メモリ使用量を約1/3に縮小できた。

#### ratingsデータ整形
データに重複があるかどうかを調べる。

In [8]:
ratings.duplicated().sum()

np.int64(3329887)

重複が存在する事が判明。今後の集計作業のために、これは削除しておく。

`drop_duplicates`メソッドで、重複した行を削除する事が出来る。

In [9]:
ratings.drop_duplicates()

,userID,animeID,rating
0,1,1,10
1,1,2,10
2,1,3,7
3,1,4,10
4,1,5,10
...,...,...,...
148170491,1774522,394,7
148170492,1774522,422,7
148170493,1774522,1018,7
148170494,1774522,590,6


In [10]:
ratings.shape

(148170496, 3)

#### animesデータ整形

まずは、こちらにも重複が無いか念のため調べておく。

In [11]:
animes.duplicated().sum()

np.int64(0)

重複は存在しない。

In [12]:
animes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20237 entries, 0 to 20236
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   animeID   20237 non-null  int16   
 1   type      20237 non-null  category
 2   year      20237 non-null  object  
 3   episodes  20237 non-null  int16   
 4   sequel    20237 non-null  bool    
dtypes: bool(1), category(1), int16(2), object(1)
memory usage: 277.0+ KB


まずは`year`列をクリーニングする。

"?"が列に含まれているので…

In [13]:
(animes["year"] == "?").sum()

np.int64(136)

np.nanを入れてしまう

In [14]:
animes.loc[animes["year"] == "?", "year"] = np.nan

In [15]:
animes["year"].isna().sum()

np.int64(136)

では、データ型を変換してみようと思う。欠損値を受け入れてくれる、pandasの拡張データ型を使用する。

In [16]:
animes["year"] = animes["year"].astype("Int16")

ここまでで、2つのテーブルのメモリ使用量を確認する。<br>
特に、`animes`テーブルは、大幅に減らすことに成功した。<br>
これなら、データの結合にも耐えられるのではないか。

In [17]:
display_MB(animes)

'0.17 MB'

In [18]:
display_GB(ratings)

'0.97 GB'

## データの結合を実行

In [19]:
merged = pd.merge(ratings, animes, on="animeID")

In [20]:
merged

,userID,animeID,rating,type,year,episodes,sequel
0,1,1,10,MOVIE,2004,1,False
1,1,2,10,TV,2006,37,False
2,1,3,7,TV,2013,10,False
3,1,4,10,TV,2012,12,False
4,1,5,10,TV,2012,25,False
...,...,...,...,...,...,...,...
148170491,1774522,394,7,TV,2007,24,False
148170492,1774522,422,7,TV,2007,12,False
148170493,1774522,1018,7,TV,2012,12,False
148170494,1774522,590,6,TV,2010,12,False


In [21]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148170496 entries, 0 to 148170495
Data columns (total 7 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userID    int32   
 1   animeID   int16   
 2   rating    int8    
 3   type      category
 4   year      Int16   
 5   episodes  int16   
 6   sequel    bool    
dtypes: Int16(1), bool(1), category(1), int16(2), int32(1), int8(1)
memory usage: 1.9 GB


何とか、2GB程度のデータ量で済ませる事が出来た。<br>
このデータは今後の分析で活用するため、再利用の為に、pickle形式で保存する。

In [22]:
merged.to_pickle(Path("..", "data", "processed", "merged.pkl"))

## 参照用テーブルの作成
別途、参照用のテーブルを作成する。<br>
さらに、今後の分析をスムーズに実行するため、このページであらかじめ必要なクリーニング作業を終わらせておく。

In [23]:
animes = pd.read_csv(
    Path("..", "data", "raw", "animes.csv"),
    usecols=[
        'animeID', 'title', 'alternative_title', 'genres',
        'genres_detailed', 'image_url', 'score', 'episodes'
    ],
    dtype={
        'animeID': "int16",
        'episodes': "int16"
    }
)

In [24]:
animes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20237 entries, 0 to 20236
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   animeID            20237 non-null  int16 
 1   title              20237 non-null  object
 2   alternative_title  8676 non-null   object
 3   score              20237 non-null  object
 4   episodes           20237 non-null  int16 
 5   image_url          20237 non-null  object
 6   genres             20237 non-null  object
 7   genres_detailed    20237 non-null  object
dtypes: int16(2), object(6)
memory usage: 1.0+ MB


重複を確認

In [25]:
animes.duplicated().sum()

np.int64(0)

`score`列にも不要な値が含まれている事が分かっているので、クリーニングした後、pickle形式で別途保存する。

In [26]:
(animes["score"] == "?").sum()

np.int64(593)

np.nanを入れてしまう

In [27]:
animes.loc[animes["score"] == "?", "score"] = np.nan

In [28]:
(animes["score"] == "?").sum()

np.int64(0)

In [29]:
animes["score"].isna().sum()

np.int64(593)

浮動小数型に変換

In [30]:
animes["score"] = animes["score"].astype("float32")

別途保存。

In [31]:
animes.to_pickle(Path("..", "data", "processed", "animes.pkl"))